In [60]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.19.0


In [61]:
!ls $AZURE_MODEL_DIR

data	      deploy.ipynb.amltemp  scripts		    train.py
data.zip      giftboxcnn.pkl	    training.ipynb
deploy.ipynb  score.py		    training.ipynb.amltemp


In [68]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from azureml.core.model import Model
import logging
import tensorflow as tf

logging.basicConfig(level=logging.DEBUG)

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    # model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'giftboxcnn.pkl')
    model_path = Model.get_model_path(model_name='giftboxcnn2')
    print(Model.get_model_path(model_name='giftboxcnn2'))
    model = tf.keras.models.load_model(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Overwriting score.py


In [69]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "custom",  "method" : "keras"}, 
                                               description='Predict items with keras')

In [74]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies

ws = Workspace.from_config()
model = Model(ws, 'giftboxcnn2')

cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults', 'joblib', 'tensorflow'])

myenv = Environment("giftbox")
myenv.python.conda_dependencies = cd
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service_name = 'giftbox'
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 36950880-fbde-413e-b92e-50dd4f0ff98f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, NameError: name 'tf' is not defined, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, NameError: name 'tf' is not defined, please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 36950880-fbde-413e-b92e-50dd4f0ff98f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, NameError: name 'tf' is not defined, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, NameError: name 'tf' is not defined, please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 36950880-fbde-413e-b92e-50dd4f0ff98f\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"AciDeploymentFailed\",\n  \"message\": \"Aci

In [75]:
print(ws.webservices)

# Choose the webservice you are interested in

from azureml.core import Webservice

service = Webservice(ws, service_name)
print(service.get_logs())

{'giftbox': AciWebservice(workspace=Workspace.create(name='HTN', subscription_id='781c9a35-470f-41aa-8e64-1437641d34ec', resource_group='HTN'), name=giftbox, image_id=None, compute_type=None, state=ACI, scoring_uri=None, tags=None, properties={'data': 'custom', 'method': 'keras'}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})}
/usr/sbin/nginx: /azureml-envs/azureml_dfac7b83a9c2996a0077811f772ac202/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dfac7b83a9c2996a0077811f772ac202/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dfac7b83a9c2996a0077811f772ac202/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dfac7b83a9c2996a0077811f772ac202/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azur

In [ ]:
print(service.scoring_uri)